In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('D:/Machine Learning Projects/Song Recommendation System/Spotify data/spotify_millsongdata.csv')

In [11]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [12]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [13]:
df.shape

(57650, 4)

In [14]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [15]:
df = df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [16]:
df.head(10)

,artist,song,text
0,Point Of Grace,Love Enough,"On your own, \r\nAll alone and crying \r\nA ..."
1,Lil Wayne,Hot Revolver,"Young Mullah, baby \r\nHmm, Young Mullah, bab..."
2,Frank Sinatra,Blue Lace,"Blue lace, her gown was blue lace, \r\nShe wo..."
3,Morrissey,At Last I Am Born,At last I am born \r\nHistorians note \r\nI ...
4,Chuck Berry,Nadine (Is It You?),"I got on a city bus and found a vacant seat, ..."
5,Andrea Bocelli,Because,Because you come to me with naught save love ...
6,Beach Boys,I Can Hear Music,Ahhhhhh oooooo \r\n \r\nThis is the way \r\...
7,Hollies,Feet On The Ground,If we speak of lovin' \r\nWill it come from o...
8,Perry Como,"Goodnight, Sweet Jesus","Goodnight, Sweet Jesus, \r\nGoodnight, goodni..."
9,Keith Urban,But For The Grace Of God,I can hear the neighbors \r\nThey're arguin' ...


In [28]:
df['text'][0]

"on your own , all alon and cri a wound soul on an island in the blue love 's requir but you 're too tire from tri do n't give up there 's love enough for you [ choru ] there 's love enough for a broken heart love enough for anoth start love enough in the father 's arm for you when it feel like the tear wo n't end when lone is your onli friend there 's love enough love enough for you on hi own allon and die he gave up everyth for me and you hear your heart it 's time to start reli on the one who had love enough for you [ choru ] like a cup that 's run over like a well that wo n't run dri from the storm a constant cover for a heart , the sole suppli choru ( 2x ) love enough for you love enough , love enough love enough for me love enough for you"

In [24]:
df.shape

(5000, 3)

Text Cleaning / Preprocessing

In [25]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [26]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [27]:
df['text'] = df['text'].apply(lambda x:tokenization(x))

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [31]:
similarity[0]

array([1.        , 0.08017913, 0.0698899 , ..., 0.01582054, 0.03323545,
       0.04492595])

In [33]:
df[df['song'] == 'But For The Grace Of God']

,artist,song,text
9,Keith Urban,But For The Grace Of God,i can hear the neighbor they 're arguin ' agai...


In [34]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs=[]
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [35]:
recommendation('But For The Grace Of God')

['Hushabye',
 'Grace',
 "People, Let's Stop The War",
 'Fresh Air',
 'There But For The Grace Of God',
 'She Got Me',
 'God Love You',
 'Your Saving Grace',
 'By Your Side',
 'Lord, In Your Love',
 "I'm Singing",
 'God Is Good [Live]',
 'Broken Vessels',
 'At Last (Live)',
 'I Get Around',
 'I Live To Know You',
 'Almost A Lover',
 'One Last Try',
 'Christopher Robin Is Saying His Prayers',
 'The Gospel Of Grace']

In [36]:
import pickle
pickle.dump(similarity,open('similarity_song.pkl','wb'))
pickle.dump(df,open('dfs.pkl','wb'))